In [1]:
from __future__ import print_function
import os.path
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

## stuff that's rather hard to find from documentation
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.install', 'https://www.googleapis.com/auth/drive', \
'https://www.googleapis.com/auth/docs', 'https://www.googleapis.com/auth/drive.readonly']

In [3]:
cwd = os.getcwd()

In [5]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(cwd+'/credentials/token.json'):
    creds = Credentials.from_authorized_user_file(cwd+'/credentials/token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(cwd+'/credentials/oauth_credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(cwd+'/credentials/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('drive', 'v3', credentials=creds)

In [1]:
NCN_master_id = '1OjlcHKkyKe9Uw_3iTFxIUafdtD1NNXKC'
sf_import_master_id = '1Hg6UQmf5z2ZRqOwVnOHMqjLwZkZN-jOh'
tbd_import_id = '1gIbrfPnNs8TZrhTarhWOaCl8Bts5PNX8'
failure_id = '1WMU4A6IMQhtIZ4rehpFiKiyllc718U_1'
success_id = '1lf2vTUWiZHLEhfzM8m3Tym4j9UGgdhMZ'

In [42]:
# common mimeTypes I'll need to use that can be hard to find 
# google docs native spreadsheet format: 'application/vnd.google-apps.spreadsheet' 
# xlsx (new excel format): 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' 
# xls (old excel format): 'application/vnd.ms-excel' 
# csv: 'text/csv' 
# https://en.wikipedia.org/wiki/Media_type 

In [43]:
## file downloading
# 1. search for all files with parent directory - tbd_import_id
# 2. import them sequentially

In [34]:
tbd_imp_files = service.files().list(q="'"+tbd_import_id + "' in parents",
                                      spaces='drive',
                                      fields='nextPageToken, files(id, name)').execute()

In [35]:
tbd_imp_files.get('files')

[{'id': '1H-l-e8g0o3N9L7S5dJCOs33_wJ2GENiY', 'name': 'test2.csv'}]

In [36]:
tbd_imp_ids = [] 
tbd_imp_fnames = [] 
for temp_file_dict in tbd_imp_files['files']:
    tbd_imp_ids.append(temp_file_dict['id']) 
    tbd_imp_fnames.append(temp_file_dict['name']) 

In [37]:
import io 
import shutil

In [39]:
for temp_index,f_tbd_id in enumerate(tbd_imp_ids):
    request = service.files().get_media(fileId=f_tbd_id) #,mimeType='text/csv') #if not .csv, then do .export()
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))
        
    # The file has been downloaded into RAM, now save it in a file
    # https://stackoverflow.com/questions/60111361/how-to-download-a-file-from-google-drive-using-python-and-the-drive-api-v3
    fh.seek(0)
    with open(cwd+'/To_Import/'+tbd_imp_fnames[temp_index], 'wb') as f:
        shutil.copyfileobj(fh, f) #, length=131072)    

Download 100%.


In [40]:
import pickle as pk

In [41]:
imports_need = True
with open('file_status','wb') as f:
    pk.dump(imports_need,f)

In [ ]:
with open(cwd+'/To_Import/gdrive_ids','wb') as f:
    pk.dump(tbd_imp_files,f)
# tbd_imp_files